## Train Cartoon GAN

Jupyter notebook version of the ```src.networks.train.py``` file.
This file contains all cells to train the model from scratch.

#### Configure drive (only if needed)

In [ ]:

from google.colab import drive
import os
drive.mount("/content/drive")
!cd ..
!ls
PROJECT_DIRECTORY = "drive/My Drive/DeepL/"
os.chdir(PROJECT_DIRECTORY)

!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data
data  log_file_test.log  papers  requirements.txt  src	weights


In [ ]:
!ls
!pip install -r requirements.txt

data  log_file_test.log  papers  requirements.txt  src	weights
  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


  Using cached pandas-1.1.5-cp37-cp37m-manylinux1_x86_64.whl (9.5 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
import torch
import torch.optim as optim


cuda = torch.cuda.is_available()
print(cuda)

if cuda:
  print(torch.cuda.get_device_name(0))
  !nvidia-smi

device = "cuda" if cuda else "cpu"

True
Tesla P100-PCIE-16GB
Mon Dec 27 13:42:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

In [ ]:
import logging

logging.basicConfig(filename="log_file_test.log",
  filemode='a',
  format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
  datefmt='%H:%M:%S',
  level=logging.DEBUG
)

#### Main parameters

Main (pre-)training parameters are defined here.
It is easier to change some parameters in `.ipynb` file instead of `config.py` while using Google colaboratory.

In [ ]:
from src.models.parameters import CartoonGanParameters

BATCH_SIZE = 8

pretraining_parameters = CartoonGanParameters(
    epochs=10,
    gen_lr=0.0002,
    disc_lr=0.0002,
    batch_size=BATCH_SIZE,
    conditional_lambda=10,
    gen_beta1=0.5,
    gen_beta2=0.999,
    disc_beta1=0.5,
    disc_beta2=0.999
)

### Prepare data

In [10]:
from src.extraction.__main__ import *

# Extract the data if needed

# create_all_frames_csv()
# create_all_images_csv()
# create_train_test_frames()
# create_train_test_images()

In [ ]:
from src.dataset.dataset_cartoon import CartoonDataset
from src.dataset.dataset_pictures import PicturesDataset
from src.preprocessing.preprocessor import Preprocessor
from torch.utils.data import DataLoader

In [ ]:
preprocessor = Preprocessor(size=256)

pictures_dataset = PicturesDataset(
    train=True,
    transform=preprocessor.picture_preprocessor(),
    size=None # keep all data
)

train_pictures_loader = DataLoader(
    dataset=pictures_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    # drop last incomplete batch
    drop_last=True,
    num_workers=2
)

### Load model

In [ ]:
from src.models.cartoon_gan import CartoonGan

cartoon_gan = CartoonGan(
    nb_resnet_blocks=8,
    nb_channels_picture=3,
    nb_channels_cartoon=3,
    nb_channels_1_h_l_gen=64,
    nb_channels_1_h_l_disc=32
)

In [ ]:
pretrained_gen = os.path.join("weights", "pretrained", "pretrained_gen_6.pkl")
pretrained_disc = os.path.join("weights", "pretrained", "pretrained_disc_6.pkl")

cartoon_gan.load_model(pretrained_gen, pretrained_disc)

### Pretrain model

In [ ]:

cartoon_gan.pretrain(
    pictures_loader=train_pictures_loader,
    parameters=pretraining_parameters
)

  8%|▊         | 305/4045 [03:06<38:05,  1.64it/s]


KeyboardInterrupt: ignored

In [ ]:
!ls
ROOT_FOLDER = os.path.join(os.path.abspath("src"), "..")

cartoon_gan.save_model(
    os.path.join(ROOT_FOLDER, "weights", "pretrained", "pretrained_gen_6.pkl"),
    os.path.join(ROOT_FOLDER, "weights", "pretrained", "pretrained_disc_6.pkl")
)

data  log_file_test.log  papers  requirements.txt  src	weights
